In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from my_setting import my_pd_setting, my_np_setting, my_sns_setting, data_path
from my_stat import missing
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, learning_curve, GridSearchCV
from sklearn.svm import SVC
my_np_setting()
my_sns_setting()
my_pd_setting()
filepath = r'C:\Users\86199\Documents\Datas\Mechine_learing\book1'
data_path(filepath)

numpy的版本为1.20.3
seaborn的版本为: 0.11.2
pandas的版本为: 1.3.4
当前数据路径为: C:\Users\86199\Documents\Datas\Mechine_learing\book1


['boston_house_prices.csv',
 'train_ex3.csv',
 'wdbc.csv',
 'wdbc.txt',
 'wine.csv',
 'wine.txt']

In [3]:
df = pd.read_csv('train_ex3.csv')
df.head()


,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
print(df['Class label'].unique())
print(np.max(df['Class label']))
print('类别变量各个类的数量')
print(df['Class label'].value_counts())

[1 2 3]
3
类别变量各个类的数量
2    71
1    59
3    48
Name: Class label, dtype: int64


##### 查看数据类型

In [5]:
dtype = pd.DataFrame(df.dtypes, columns=['dtypes'])
dtype.reset_index(inplace=True)
dtype.rename(columns={'index':'var_names'}, inplace=True)
dtype

,var_names,dtypes
0,Class label,int64
1,Alcohol,float64
2,Malic acid,float64
3,Ash,float64
4,Alcalinity of ash,float64
5,Magnesium,int64
6,Total phenols,float64
7,Flavanoids,float64
8,Nonflavanoid phenols,float64
9,Proanthocyanins,float64


In [6]:
X= df.drop(columns=['Class label']).values
# y = df[['Class label']].values # 会发生 DataConversionWarning警告
y= df.loc[:,['Class label']].values.ravel() # ravel可以避免DataConversionWarning
la = LabelEncoder()
y = la.fit_transform(y)
print(f'''y的形状为: {y.shape}''')
print(f'''y包含的类标签为:  {la.classes_}''')
print(f'''列表[3,1,2]的编码情况为:   {la.transform([3,1,2])}''')


y的形状为: (178,)
y包含的类标签为:  [1 2 3]
列表[3,1,2]的编码情况为:   [2 0 1]


##### 切分数据集——获得训练集和最终测试集

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1, stratify=y)
print(f'''X_train的形状: {X_train.shape}''')
print(f'''X_test的形状: {X_test.shape}''')
print(f'''y_train的形状: {y_train.shape}''')
print(f'''y_test的形状: {y_test.shape}''')

X_train的形状: (142, 13)
X_test的形状: (36, 13)
y_train的形状: (142,)
y_test的形状: (36,)


##### 流水线——单次

In [8]:
pipe_lr = make_pipeline(
    StandardScaler(),
    PCA(n_components=7),
    LogisticRegression(random_state=1, solver='newton-cg', multi_class='multinomial')
)
pipe_lr.fit(X_train,y_train)
y_pred = pipe_lr.predict(X_test)
print(f'精度为: {pipe_lr.score(X_test, y_test)}')

精度为: 0.9722222222222222


##### K折、流水线

In [9]:
# np.bincount()
# print(np.bincount([0,1,2,3,1,2]))
a = [0, 1, 1, 3, 2, 1, 7]
print(f'''原始输入: {a}''')
print(f'''各个元素计数输出: {np.bincount(a)}''')
print(f'''各个元素排序输出: {np.sort(a)}''')

原始输入: [0, 1, 1, 3, 2, 1, 7]
各个元素计数输出: [1 3 1 1 0 0 0 1]
各个元素排序输出: [0 1 1 1 2 3 7]


In [10]:
kfolds = StratifiedKFold(n_splits=10).split(X_train, y_train)
# kfolds = StratifiedKFold(n_splits=5, random_state=1, shuffle=True) 
# 返回索引,注意报错TypeError,'StratifiedKFold' object is not iterable, 因为你没有切分数据集
scores = []
for k, (train_index, test_index) in enumerate(kfolds, start=1):
    pipe_lr.fit(X_train[train_index], y_train[train_index])
    score = pipe_lr.score(X_train[test_index], y_train[test_index])
    scores.append(score)
    print(f'''Fold: {k}; 训练时每个类数量{np.bincount(y_train[train_index])}; 训练得分: {score} ''')
print(f'''最终训练得分: {np.mean(scores)-np.std(scores)}~{np.mean(scores)+np.std(scores)}''')

Fold: 1; 训练时每个类数量[42 51 34]; 训练得分: 1.0 
Fold: 2; 训练时每个类数量[42 51 34]; 训练得分: 1.0 
Fold: 3; 训练时每个类数量[42 52 34]; 训练得分: 1.0 
Fold: 4; 训练时每个类数量[42 52 34]; 训练得分: 1.0 
Fold: 5; 训练时每个类数量[42 52 34]; 训练得分: 1.0 
Fold: 6; 训练时每个类数量[43 51 34]; 训练得分: 0.9285714285714286 
Fold: 7; 训练时每个类数量[43 51 34]; 训练得分: 1.0 
Fold: 8; 训练时每个类数量[43 51 34]; 训练得分: 0.9285714285714286 
Fold: 9; 训练时每个类数量[42 51 35]; 训练得分: 0.9285714285714286 
Fold: 10; 训练时每个类数量[42 51 35]; 训练得分: 0.9285714285714286 
最终训练得分: 0.9364358608173833~1.0064212820397598


##### cross_val_score

In [11]:
scores = cross_val_score(
    estimator=pipe_lr, X=X_train, y=y_train, cv=10, n_jobs=2
)
print(f'''得分情况: {scores}''')
print(f'''得分区间: [{np.mean(scores)-np.std(scores)}, {np.mean(scores)+np.std(scores)}]''')

得分情况: [1.         1.         1.         1.         1.         0.92857143
 1.         0.92857143 0.92857143 0.92857143]
得分区间: [0.9364358608173833, 1.0064212820397598]


##### 网格搜索最优参数，SVM模型

In [19]:
pipe_svc = make_pipeline(
    StandardScaler(),
    SVC(random_state=1)
)
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

param_grid = [{'svc__C': param_range, 
               'svc__kernel': ['linear']},
              {'svc__C': param_range, 
               'svc__gamma': param_range, 
               'svc__kernel': ['rbf']}]

gs = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=10,
                  return_train_score=True,
                  n_jobs=-1)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.9861904761904763
{'svc__C': 1.0, 'svc__gamma': 0.1, 'svc__kernel': 'rbf'}


In [31]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, make_scorer
scoring= {
    # 'f1':'f1',
    # 'roc_auc':'roc_auc',
    'accuracy':'accuracy'
}
gs = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring=scoring, 
                  refit='accuracy',
                  cv=10,
                  return_train_score=True,
                  n_jobs=-1)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.9861904761904763
{'svc__C': 1.0, 'svc__gamma': 0.1, 'svc__kernel': 'rbf'}


In [32]:
gs.cv_results_ # 字典形式的得分
# score_train = gs.cv_results_['mean_train_score']
# score_test = gs.cv_results_['mean_test_score']
# print(np.max(score_train))
# print(np.max(score_test))

{'mean_fit_time': array([0.00289235, 0.00269322, 0.00259316, 0.00199499, 0.00199482,
        0.00239375, 0.00229385, 0.00219426, 0.00279233, 0.00299191,
        0.00269284, 0.00289197, 0.00329099, 0.00319159, 0.00418856,
        0.00418918, 0.00309172, 0.00289221, 0.00279272, 0.00299242,
        0.00468757, 0.00299203, 0.00478716, 0.00797844, 0.00309153,
        0.00388944, 0.00299191, 0.00259283, 0.00348973, 0.00643392,
        0.00613463, 0.00478725, 0.00279298, 0.00329103, 0.00309172,
        0.00438848, 0.00349069, 0.00344226, 0.00488677, 0.00498667,
        0.00304267, 0.00319064, 0.00249333, 0.00219409, 0.0043391 ,
        0.00404069, 0.0046386 , 0.00498652, 0.00463836, 0.00693245,
        0.00209436, 0.00259299, 0.0035902 , 0.00598388, 0.0048866 ,
        0.00588448, 0.00299196, 0.00229397, 0.00279264, 0.0029922 ,
        0.00379   , 0.00488696, 0.00528567, 0.00558529, 0.00249357,
        0.00229402, 0.00229394, 0.00249317, 0.00378983, 0.00568471,
        0.00538547, 0.00458794]